## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-06-21-51-59 +0000,nypost,Air India’s midair emergency sparks new alarm ...,https://nypost.com/2025/10/06/business/air-ind...
1,2025-10-06-21-51-24 +0000,bbc,Shoe thrown at India's top judge in religious row,https://www.bbc.com/news/articles/c80g4d5epm8o...
2,2025-10-06-21-49-13 +0000,nyt,"Supreme Court, for Now, Rejects Google Bid to ...",https://www.nytimes.com/2025/10/06/us/politics...
3,2025-10-06-21-48-46 +0000,bbc,Judge blocks Trump from sending troops from Ca...,https://www.bbc.com/news/articles/c740elm70z7o...
4,2025-10-06-21-45-21 +0000,wapo,The Senate is back with no shutdown deal in sight,https://www.washingtonpost.com/business/2025/1...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,48
5,new,22
13,judge,15
158,gaza,14
17,court,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
58,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...,107
64,2025-10-06-19-54-25 +0000,nypost,Judge rejects Trump admin’s bid to delay Kilma...,https://nypost.com/2025/10/06/us-news/judge-re...,106
18,2025-10-06-21-29-00 +0000,wsj,President Trump signaled a willingness to stri...,https://www.wsj.com/politics/policy/trump-open...,105
229,2025-10-06-08-50-56 +0000,wapo,Judge deals Trump new setback in plans to depl...,https://www.washingtonpost.com/nation/2025/10/...,104
92,2025-10-06-18-34-32 +0000,nypost,Trump says Gaza peace talks going ‘very well’ ...,https://nypost.com/2025/10/06/us-news/trump-pu...,100


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
58,107,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...
71,83,2025-10-06-19-34-39 +0000,cbc,Bari Weiss named editor-in-chief of CBS News u...,https://www.cbc.ca/news/entertainment/bari-wei...
114,57,2025-10-06-17-44-15 +0000,wapo,Prime minister of France resigns hours after n...,https://www.washingtonpost.com/world/2025/10/0...
274,45,2025-10-05-23-07-07 +0000,bbc,Gisèle Pelicot to face one of her rapists in c...,https://www.bbc.com/news/articles/cdx2d9lynk1o...
80,43,2025-10-06-19-00-14 +0000,nyt,Israel and Hamas Meet in Egypt to Discuss Gaza...,https://www.nytimes.com/2025/10/06/world/middl...
211,36,2025-10-06-10-00-00 +0000,nypost,"These Americans died fighting in Ukraine, now ...",https://nypost.com/2025/10/06/world-news/paren...
60,33,2025-10-06-19-56-20 +0000,nypost,NYC man turns up dead in car after parents rep...,https://nypost.com/2025/10/06/us-news/nyc-man-...
77,33,2025-10-06-19-11-11 +0000,bbc,US Supreme Court rejects Ghislaine Maxwell app...,https://www.bbc.com/news/articles/cn83e6q7lq4o...
128,33,2025-10-06-16-48-36 +0000,bbc,Sudan militia leader convicted of war crimes d...,https://www.bbc.com/news/articles/cly1egd5d0vo...
207,31,2025-10-06-10-07-35 +0000,cbc,"American, Japanese scientists win Nobel prize ...",https://www.cbc.ca/news/world/nobel-sweden-med...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
